# [GALAXY] - Photometry Checker

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook should be run with the <span style="font-family: 'Ariel', monospace;">stenv</span> environment
    where the <span style="font-family: 'Ariel', monospace;">regions</span> package has been added if not in the current
    version of <span style="font-family: 'Ariel', monospace;">stenv</span>.
</div>

This notebook is designed to check the photometry between the Hubble Pipeline DRCs and
this pipeline's DRCs. Predominantly, this checks for *systematic* offsets in the data. That is,
some individual sources may have high offset due to differences in how CRs are processed, but most
sources should exhibit low difference in the measured photometry.

## Prerequisites

Note that this notebook depends on a
[DS9](https://sites.google.com/cfa.harvard.edu/saoimageds9/home) region file
called `../DS9/PhotometryCheckerSources.reg` to be present to run.
This region file should define a set of sources (using any region that defines a center such
as a point or a circle) that will have their photometry compared between this pipeline and the
*HST* pipeline. For simplicity, this could be a symbolic link to the region file created by the
[GAIA Notebook](../../../../Data/GAIA/GAIA_Downloader.ipynb).

## Imports

In [ ]:
# Python Imports
import os
from pathlib import Path
from glob import iglob

# 3rd Party Imports
import numpy as np

# Astropy Imports
from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import photutils as phot
import regions

## Notebook Setup

In [ ]:
# Get the Notebook's Directory and ChangeDir to it
NOTEBOOK_DIR = Path(__file__).resolve().parent
os.chdir(NOTEBOOK_DIR)

In [ ]:
# Paths
TOP_LVL_DIR = Path('../../../../').resolve()
IMG_DIR     = TOP_LVL_DIR / 'Images'
MAST_DIR    = IMG_DIR / 'mastDownload/HST'
PROC_DIR    = IMG_DIR / 'ProcessedImages/HST'

## Load Data

### Load HST Pipeline Images

In [ ]:
# Load HST DRCs
hstData = {}
for fn in MAST_DIR.rglob('*dr?.fits'):

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

        # Store the Name using the filter as the dict key
        # Start the Empty List if Key does not exist
        if filt not in hstData:
            hstData[filt] = {}

        # Load the Image Data and WCS
        hstData[filt]['img'] = hduList[1].data
        hstData[filt]['wcs'] = WCS(hduList[1])

### Load My Pipeline Images

In [ ]:
# Load My DRCs
myData = {}
for fn in PROC_DIR.rglob('*dr?_irn.fits'):  # May need to change glob if NaN Inpainter Skipped

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

        # Store the Name using the filter as the dict key
        # Start the Empty List if Key does not exist
        if filt not in myData:
            myData[filt] = {}

        # Load the Image Data and WCS
        myData[filt]['img'] = hduList[1].data
        myData[filt]['wcs'] = WCS(hduList[1])

### Load Region Coordinates

In [ ]:
# Load Regions Data
ds9Regs = regions.Regions.read(PROC_DIR / 'DS9' / 'PhotometryCheckerSources.reg')

# Convert Centers to SkyCoord Array
srcCrds = SkyCoord([reg.center for reg in ds9Regs])

## Photometry

In [ ]:
# Radii
R_IN  = 0.5 * u.arcsec
R_OUT = 0.9 * u.arcsec

# Area
A_IN  = np.pi*R_IN**2
A_OUT = np.pi*(R_OUT**2 - R_IN**2)

# Setup Apertures
aper = phot.aperture.SkyCircularAperture(srcCrds, R_IN)
annu = phot.aperture.SkyCircularAnnulus(srcCrds, R_IN, R_OUT)

### Simple Aperture Photometry

In [ ]:
# Loop through HST Data
for dataDict in hstData.values():

    # Get Photometry
    aperRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        aper,
        wcs=dataDict['wcs']
    )
    annuRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        annu,
        wcs=dataDict['wcs']
    )

    # Store Results
    dataDict['flux'] = aperRes['aperture_sum'] - annuRes['aperture_sum']*(A_IN/A_OUT)

In [ ]:
# Loop through My Data
for dataDict in myData.values():

    # Get Photometry
    aperRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        aper,
        wcs=dataDict['wcs']
    )
    annuRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        annu,
        wcs=dataDict['wcs']
    )

    # Store Results
    dataDict['flux'] = aperRes['aperture_sum'] - annuRes['aperture_sum']*(A_IN/A_OUT)

In [ ]:
# Loop through Filters
for filt in hstData:

    # Print
    print(filt)
    display(
        (hstData[filt]['flux'] - myData[filt]['flux'])/hstData[filt]['flux'] << u.percent
    )